# Retrieve and check fod jobs

In [ ]:
"""
TODO:

- [ ] move functions to library
- [ ] collect other stats from test jobs

"""

In [ ]:
#    - config file: `check_fod_jobs.yml` in the same directory as this notebook

In [ ]:
# *NOTE*: run this command to clean output cell and meta data.
# $ nb-clean clean ./check_fod_jobs.ipynb 

In [ ]:
# generate html
# $ python ./convert_nb.py --filename ./check_watchdog_issue_fod_jobs.ipynb --execute

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Tuple
import yaml
import json
import os
import logging
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter

from utils import *

from typing import Iterable, Any, List, Optional, Union, Callable, TextIO, Dict, Tuple

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, Markdown, Latex

pd.set_option('max_colwidth', 800)

In [ ]:
CONFIG_FILE = "configs/check_watchdog_issue_fod_jobs.yml"

WDT_LONG_MSG = "WDT: Long-running WU warning"

In [ ]:
config = read_config(CONFIG_FILE)

In [ ]:
# print(config)

In [ ]:
def printmd(string):  ###
    display(Markdown(string))  ###

In [ ]:
# extract information from job and console response
# summary_table is list of dict
def generagte_summary_table(collected_jobs: List[Dict[str, str]]) -> List[Dict[str, str]]:
    """Generate summary table

    Parameters
    ----------
    collected_jobs : List[Dict[str, str]]
        list of dict of job info

    Returns
    -------
    List[Dict[str, str]]
        list of dict of summary
    """
    summary_table = []
    for job in collected_jobs:
        machine, date_str = fod_extract_machines_from_job(job["job_response"])
        vps = fod_extract_vps_from_console(job["console_response"], pattern=WDT_LONG_MSG)

        row = {"job_no": job["job_no"], "machine": machine, "vps": vps, "job_url": job["job_url"], "console_url": job["console_url"], "date": date_str}
        summary_table.append(row)

    return summary_table


In [ ]:
def get_vp_freq(df: pd.DataFrame) -> Counter:
    """Get VP frequency

    Parameters
    ----------
    df : pd.DataFrame
        dataframe

    Returns
    -------
    Counter
        VP frequency
    """
    vp_freq = Counter()
    for vps in df["vps"]:
        # print(vps)
        vp_freq.update(vps)
        # print(vp_freq)
    return vp_freq

In [ ]:
def get_cluster_freq(df: pd.DataFrame) -> Counter:
    """get per cluster frequency
    
    Parameters
    ----------
    df: pd.DataFrame
        dataframe
    
    Returns
    -------
    Counter
        per cluster frequency
    """

    # cluster frequency


    # df["vps"]

    # put all columns to list
    vps_list = df["vps"].tolist()

    # flatten list
    # CCV ("x.x.x"), first digit is cluster
    cluster_list = [item[0] for sublist in vps_list for item in sublist]
    # print(cluster_list)

    # frequency of each VP
    cluster_freq = Counter(cluster_list)


    # vp_freq

    # sort by frequency
    # cluster_freq.most_common()
    return cluster_freq


In [ ]:
job_str = ""
# generate string from list
for no in config['job_no']:
    job_str += str(no) + ", "

note_str = "### Loading the following jobs:\n {}".format(job_str)
printmd(note_str)
collected_jobs = fod_extract_pages(config)

In [ ]:
summary_table = generagte_summary_table(collected_jobs)

In [ ]:
note_str = "### The followings shows the jobs list, machines and VPs that watchdog occured\n Note: empty VPs is due to missing console log."
printmd(note_str)

# convert list to dataframe
df = pd.DataFrame(summary_table)
display(df[["job_no", "machine", "vps", "date"]])

In [ ]:
# check if failed jobs occurs on the same machine more frequently

note_str = "### The following shows that frequency of machines that failed jobs occured"
printmd(note_str)
df["machine"].value_counts()

In [ ]:
note_str = "### The following shows that frequency of vps that failed jobs occured"
printmd(note_str)

vp_freq = get_vp_freq(df)
vp_freq.most_common()


In [ ]:
note_str = "### The following shows that frequency of clusters that failed jobs occured"
printmd(note_str)

cluster_freq = get_cluster_freq(df)
cluster_freq.most_common()

In [ ]:
# print(df['console_url'])

In [ ]:
# print(df['job_url'])

In [ ]:
assert False

# below is test code

In [ ]:
test = collected_jobs[0]
console = test["console_response"]

# print(console)


In [ ]:
test_str = """
[12.570816 6.1.1] 
[12.571016 4.5.2] 
[12.571221 1.1.3] WDT: Long-running WU warning: par_work_wuthread__reactivate
[12.572009 0.0.0] WDT: Long-running WU warning: par_work_wuthread__reactivate
[12.572797 1.1.3] Possible last WU action: 0x820bc2f000c42586
[12.573427 0.0.0] Possible last WU action: 0x8202008000c42586
[12.574058 1.1.3] Decode WU32 header: wuid 'par_work_wuthread__reactivate' (0xc42586)
[12.574927 1.1.3] 	src  FA1:15:0[CCV1.1.3]
[12.575367 1.1.3] 	dest FA1:15:0[CCV1.1.3]
[12.571221 1.1.4] WDT: Long-running WU warning: par_work_wuthread__reactivate
[12.575367 1.1.3] 	dest FA1:15:0[CCV1.1.3]
"""

In [ ]:
test_str = console

In [ ]:
assert False

In [ ]:
import re

pattern = "WDT: Long-running WU warning"
# filtered_lines = re.search(pattern, test_str, re.DOTALL)
# print(filtered_lines.group(0))

# for matches in re.findo

# find all lines include "WDT: Long-running WU warning"
# filtered_lines = re.findall(pattern, test_str, re.DOTALL)

# find all lines include "WDT: Long-running WU warning" and show full line
filtered_lines = re.findall(".*"+pattern, test_str)
print(filtered_lines)
vps = []
for line in filtered_lines:
    # print(line)
    t = line.split(" ")
    # print(t)
    vp = t[1][:-1]
    vps.append(vp)

print(vps)

In [ ]:
df["vps"]

# put all columns to list
vps_list = df["vps"].tolist()

# flatten list
vps_list = [item for sublist in vps_list for item in sublist]
vps_list

# frequency of each VP
vp_freq = Counter(vps_list)


# vp_freq

# sort by frequency
vp_freq.most_common()


In [ ]:
# cluster frequency


df["vps"]

# put all columns to list
vps_list = df["vps"].tolist()

# flatten list
cluster_list = [item[0] for sublist in vps_list for item in sublist]
print(cluster_list)

# frequency of each VP
cluster_freq = Counter(cluster_list)


# vp_freq

# sort by frequency
cluster_freq.most_common()
